# Fine Tuning Model
#### 1. Only train added layers.
#### 2. Unfreeze some layers of base model. 

In [1]:
from keras.applications import VGG16
conv_base=VGG16(weights="imagenet",
               include_top=False,
               input_shape=(150,150,3))

Using TensorFlow backend.


In [2]:
from keras import layers
from keras import models

model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


# Freeze conv_base's params

In [3]:
print("Before freezing, trainable params num: "+str(len(model.trainable_weights)))
conv_base.trainable=False
print("After freezing, trainable params num: "+str(len(model.trainable_weights)))

Before freezing, trainable params num: 30
After freezing, trainable params num: 4


In [4]:
from keras.preprocessing.image import ImageDataGenerator
import os
train_gen=ImageDataGenerator(rescale=1/255.,
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode="nearest")
test_gen=ImageDataGenerator(rescale=1/255.)

def getGen(gen,dir_):
    return gen.flow_from_directory(dir_,
                                  target_size=(150,150),
                                  batch_size=20,
                                  class_mode="binary")
train_dir=os.path.join("small_train","train")
val_dir=os.path.join("small_train","val")
train_gener=getGen(train_gen,train_dir)
val_gener=getGen(test_gen,val_dir)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Attention! We just used VGG16 as the pretrained model, so make the learning rate small, like e-5.

In [5]:
from keras import optimizers
model.compile(
            #optimizer="rmsprop",
             optimizer=optimizers.RMSprop(lr=2e-5),
             loss="binary_crossentropy",
             metrics=["acc"])
history=model.fit_generator(train_gener,
                            steps_per_epoch=100,
                            epochs=20,
                            validation_data=val_gener,
                            validation_steps=50)

Epoch 1/20
100/100 [==============================] - 29s 286ms/step - loss: 0.6018 - acc: 0.6720 - val_loss: 0.4374 - val_acc: 0.8130
Epoch 2/20
100/100 [==============================] - 27s 269ms/step - loss: 0.4875 - acc: 0.7800 - val_loss: 0.3847 - val_acc: 0.8330
Epoch 3/20
100/100 [==============================] - 26s 265ms/step - loss: 0.4372 - acc: 0.7965 - val_loss: 0.3316 - val_acc: 0.8510
Epoch 4/20
100/100 [==============================] - 26s 264ms/step - loss: 0.4152 - acc: 0.8130 - val_loss: 0.3126 - val_acc: 0.8640
Epoch 5/20
100/100 [==============================] - 27s 268ms/step - loss: 0.3959 - acc: 0.8265 - val_loss: 0.3062 - val_acc: 0.8590
Epoch 6/20
100/100 [==============================] - 26s 264ms/step - loss: 0.3807 - acc: 0.8260 - val_loss: 0.2933 - val_acc: 0.8670
Epoch 7/20
100/100 [==============================] - 26s 264ms/step - loss: 0.3731 - acc: 0.8320 - val_loss: 0.2858 - val_acc: 0.8700
Epoch 8/20
100/100 [==============================] - 2

# Unfreeze some top layers of base model

In [6]:
conv_base.trainable=True
set_trainable=False
for layer in conv_base.layers:
    if layer.name=="block5_conv1":
        set_trainable=True
    layer.trainable=set_trainable
    

# Retrain the model

In [7]:
history=model.fit_generator(train_gener,
                           steps_per_epoch=100,
                           epochs=10,
                           validation_data=val_gener,
                           validation_steps=50)

C:\Users\test07\Anaconda3\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/10
100/100 [==============================] - 27s 271ms/step - loss: 0.3302 - acc: 0.8545 - val_loss: 0.2591 - val_acc: 0.8930
Epoch 2/10
100/100 [==============================] - 27s 268ms/step - loss: 0.3206 - acc: 0.8575 - val_loss: 0.2590 - val_acc: 0.8960
Epoch 3/10
100/100 [==============================] - 27s 267ms/step - loss: 0.3191 - acc: 0.8600 - val_loss: 0.2569 - val_acc: 0.8940
Epoch 4/10
100/100 [==============================] - 27s 268ms/step - loss: 0.2999 - acc: 0.8685 - val_loss: 0.2657 - val_acc: 0.8850
Epoch 5/10
100/100 [==============================] - 27s 268ms/step - loss: 0.3167 - acc: 0.8570 - val_loss: 0.2510 - val_acc: 0.8990
Epoch 6/10
100/100 [==============================] - 27s 268ms/step - loss: 0.3110 - acc: 0.8605 - val_loss: 0.2545 - val_acc: 0.8950
Epoch 7/10
100/100 [==============================] - 27s 267ms/step - loss: 0.2994 - acc: 0.8740 - val_loss: 0.2582 - val_acc: 0.8940
Epoch 8/10
100/100 [==============================] - 2

In [8]:
test_dir=os.path.join("small_train","test")
test_gener=getGen(test_gen,test_dir)
loss,acc=model.evaluate_generator(test_gener)

print("Test loss: "+str(loss)+". Test acc: "+str(acc))

Found 1000 images belonging to 2 classes.
Test loss: 0.251066983044. Test acc: 0.900999996662
